In [1]:
import torch

In [2]:
torch.cuda.is_available()

False

In [3]:
torch.backends.mps.is_built()

True

In [4]:
torch.backends.mps.is_available()

True

In [5]:
device = torch.device("mps")

In [6]:
import pandas as pd
import os
from torchvision.io import read_image
from torchvision.transforms import RandomHorizontalFlip, RandomVerticalFlip, RandomRotation
from torch.utils.data import Dataset
import math

labels = pd.read_csv("labels.csv", index_col=0)

class PlayerDataset(Dataset):
    def __init__(self, labels=labels, img_dir="NBA Players"):
        self.img_labels = labels
        self.img_dir = img_dir
        self.img_files = [os.path.join(path, name) for path, subdirs, files in os.walk("NBA Players") for name in files if name.endswith("jpg")]
        
        #self.filter_teams(["GSW", "WAS", "MIA", "LAL", "NYK"])
        
        self.img_labels["team_number"] = self.img_labels["team"].astype("category").cat.codes
        self.augments = [RandomHorizontalFlip(1), RandomRotation(90), RandomVerticalFlip(1)]

    def __len__(self):
        return len(self.img_files) * (len(self.augments) + 1)
    
    def filter_teams(self, teams):
        filtered = self.img_labels[self.img_labels["team"].isin(teams)]
        filtered_names = list(filtered["folder_name"])
        new_img_files = []
        for img_path in self.img_files:
            player_name = img_path.split("/")[1]
            if player_name in filtered_names:
                new_img_files.append(img_path)
        self.img_files = new_img_files
        self.img_labels = filtered.copy()
        
    
    def teams(self):
        return self.img_labels["team"].unique()

    def __getitem__(self, idx):
        augment = math.floor(idx / len(self.img_files))
        idx = idx % len(self.img_files)
            
        img_path = self.img_files[idx]
        image = read_image(img_path)
        
        player_name = img_path.split("/")[1]
        label = self.img_labels[self.img_labels["folder_name"] == player_name]["team_number"].iloc[0]
        
        if augment > 0:
            image = self.augments[augment - 1].forward(image)

        return image, int(label)

In [7]:
data = PlayerDataset()

In [8]:
teams = data.teams()

In [9]:
len(teams)

29

In [10]:
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_data, test_data = torch.utils.data.random_split(data, [train_size, test_size])

In [11]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64
EPOCHS = 75

train = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [12]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self, classes):
        super(NeuralNetwork, self).__init__()
        
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 8, 4),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(8, 32, 4),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(.1),
        )
        
        self.dense = nn.Sequential(
            nn.Linear(32 * 61 * 61, 256),
            nn.ReLU(True),
            nn.Linear(256, 64),
            nn.ReLU(True),
            nn.Linear(64, len(classes))
        )

    def forward(self, x):
        x = self.cnn(x)
        x = torch.flatten(x, 1) 
        x = self.dense(x)
        return x

In [13]:
model = NeuralNetwork(teams).to(device)

In [14]:
model

NeuralNetwork(
  (cnn): Sequential(
    (0): Conv2d(3, 8, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(8, 32, kernel_size=(4, 4), stride=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Dropout(p=0.1, inplace=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=119072, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=29, bias=True)
  )
)

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.0001)

In [16]:
size = len(train.dataset)

for epoch in range(EPOCHS):
    for batch, (images, labels) in enumerate(train): 
        optimizer.zero_grad()
        
        # Compute prediction and loss
        images = images.to(device)
        pred = model(images.float())
        
        labels = labels.to(device)
        loss = loss_fn(pred, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()

    loss, current = loss.item(), batch * len(images)
    print(f"loss: {loss:>7f}  [{epoch}]")

loss: 3.275349  [0]
loss: 3.350127  [1]
loss: 3.284935  [2]
loss: 3.323076  [3]
loss: 3.247058  [4]
loss: 3.272078  [5]
loss: 3.206850  [6]
loss: 3.083879  [7]
loss: 3.248178  [8]
loss: 3.092091  [9]
loss: 3.140833  [10]
loss: 3.153762  [11]
loss: 3.119146  [12]
loss: 2.875666  [13]
loss: 2.924116  [14]
loss: 3.051661  [15]
loss: 2.952518  [16]
loss: 2.769735  [17]
loss: 2.966228  [18]
loss: 2.626917  [19]
loss: 2.840526  [20]
loss: 2.368796  [21]
loss: 2.693542  [22]
loss: 2.516144  [23]
loss: 2.529542  [24]
loss: 2.368105  [25]
loss: 2.187574  [26]
loss: 2.325638  [27]
loss: 1.804900  [28]
loss: 1.863764  [29]
loss: 1.937640  [30]
loss: 1.898329  [31]
loss: 2.126166  [32]
loss: 1.527549  [33]
loss: 1.842572  [34]
loss: 1.920481  [35]
loss: 1.559677  [36]
loss: 1.551955  [37]
loss: 1.405977  [38]
loss: 1.444561  [39]
loss: 1.196951  [40]
loss: 0.868281  [41]
loss: 1.144575  [42]
loss: 1.152159  [43]
loss: 1.203741  [44]
loss: 1.719996  [45]
loss: 0.911502  [46]
loss: 1.037990  [47]
lo

In [17]:
all_preds = list()
all_labels = list()

with torch.no_grad():
    for batch, (images, labels) in enumerate(test):
        
        images = images.to(device)
        outputs = model(images.float())
        
        _, preds = torch.max(outputs.data, 1)
        
        all_labels.append(labels)
        all_preds.append(preds)

In [18]:
import numpy as np

preds = np.concatenate([p.cpu().numpy() for p in all_preds])
labels = np.concatenate([p.cpu().numpy() for p in all_labels])

In [19]:
((preds == labels).sum()) / len(labels)

0.11915767847971238

In [20]:
labels

array([ 6, 17, 11, ..., 12, 18, 17])